In [12]:
import tensorflow as tf
from glob import glob
from typing import List,Tuple
from data_loader import dataset_loader
print(tf.__version__)
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2.4.1


In [17]:
ds_loader = dataset_loader(tfrec_dir=r'D:\tf_records',batch_size=8)
ds_train,ds_valid,ds_test= ds_loader.build_dataset()

Building Dataset.......

((TensorSpec(shape=(None, 1536, 1536), dtype=tf.int8, name=None),
  TensorSpec(shape=(None, None), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 25, 2), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 25, 2), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, None), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 25, 2), dtype=tf.float32, name=None)),
 TensorSpec(shape=(None, 25, 2), dtype=tf.float32, name=None))


In [18]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow.keras.backend as K


def euclidean_distance_loss(y_true, y_pred):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow tensor
    :param y_p red: TensorFlow tensor of the same shape as y_true
    :return: float
    """
    #original euclidean distance loss =  K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))
    loss = K.mean(K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1)))
    return loss


#(example["grid_map"],example["grid_org_res"],example["left_bnd"],example["right_bnd"],example#["car_odo"],example["init_path"])

def nn():

    # Grid Map input
    ip_gridmap = layers.Input(shape=(1536,1536,1))

    #CNN - branch1
    x_A = layers.Conv2D(16,kernel_size=7,strides=2)(ip_gridmap)
    x_A = layers.BatchNormalization()(x_A)
    x_A = layers.ReLU()(x_A)
    x_A = layers.AvgPool2D(pool_size=(4,4))(x_A)

    x_A = layers.Conv2D(32,kernel_size=5,strides=2)(x_A)
    x_A = layers.BatchNormalization()(x_A)
    x_A = layers.ReLU()(x_A)
    x_A = layers.AvgPool2D(pool_size=(4,4))(x_A)

    
    x_A = layers.Conv2D(64,kernel_size=3,strides=2)(x_A)
    x_A = layers.BatchNormalization()(x_A)
    x_A = layers.ReLU()(x_A)
    x_A = layers.AvgPool2D(pool_size=(2,2))(x_A)


    x_A = layers.Flatten()(x_A)


    # Other inputs
    ip_grid_org_res = layers.Input(shape=(3,))
    ip_left_bnd = layers.Input(shape=(25,2))
    ip_right_bnd = layers.Input(shape=(25,2))
    ip_car_odo = layers.Input(shape=(3,))
    ip_init_path = layers.Input(shape=(25,2))
    #branch 2
    x_B = layers.Conv1D(2,kernel_size=3,padding='same')(ip_init_path)
    #branch 3
    x_C = layers.Conv1D(2,kernel_size=3,padding='same')(ip_left_bnd)
    #branch 4
    x_D = layers.Conv1D(2,kernel_size=3,padding='same')(ip_right_bnd)
    # branch 5
    conc_grid_orgres_car_odo = layers.concatenate([ip_grid_org_res,ip_car_odo])


    reshape_init_path = layers.Reshape((50,))(x_B)
    reshape_left_bnd = layers.Reshape((50,))(x_C)
    reshape_right_bnd = layers.Reshape((50,))(x_D)

    
    #concatenate feature
    concat_feat = layers.concatenate([x_A, reshape_init_path, reshape_left_bnd, reshape_right_bnd, conc_grid_orgres_car_odo])

    #concat_feat = layers.Lambda(lambda x: tf.expand_dims(x, -1))(concat_feat)

    #FC
    #output = layers.Convolution1D(50,kernel_size=3,padding='valid',activation='relu')(concat_feat)
    
    output = layers.Dense(50, activation='linear')(concat_feat)

    output = layers.Dropout(0.6)(output)
    
    output = layers.Dense(50, activation='linear')(output)
    
    output = layers.Reshape((25,2))(output)
    
    nn_fun = keras.models.Model(inputs = [ip_gridmap,ip_grid_org_res,ip_left_bnd, ip_right_bnd, ip_car_odo, ip_init_path], outputs= output)
    
    return nn_fun

In [19]:
pp_model = nn()
opt = tf.keras.optimizers.Adam()
pp_model.compile(optimizer=opt,loss=euclidean_distance_loss, metrics='accuracy')
pp_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 1536, 1536,  0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 765, 765, 16) 800         input_19[0][0]                   
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (None, 765, 765, 16) 64          conv2d_9[0][0]                   
__________________________________________________________________________________________________
re_lu_9 (ReLU)                  (None, 765, 765, 16) 0           batch_normalization_9[0][0]      
____________________________________________________________________________________________

In [20]:
history = pp_model.fit(ds_train,epochs=1,validation_data=ds_valid)

ResourceExhaustedError:  OOM when allocating tensor with shape[8,16,765,765] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/model_3/average_pooling2d_9/AvgPool/AvgPoolGrad (defined at <ipython-input-20-882490cdecf5>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_6688]

Function call stack:
train_function
